#Proyecto Parte 2 - Krycia Castillo
##Aplicación de redes Neuronales


#Exportar Librerías

In [67]:
#Instalación e importación de librerías:
!pip install pyreadstat
import pyreadstat
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

#Carga de datos

In [68]:
# Cargando los datos para aplicación de redes neuronales, se debe colocar la ruta de aceso donde se encuentra el archivo a cargar. Tomar en cuenta que el archivo se obtuvo por medio de el notebook de R donde se realizó el análisis de árbol de decisión y random forest. El último chunk del notebook corresponde a la generación de este archivo. Para leer el archivo en google colab se puede vincular con google drive o cargar el archivo directo y posteriormente especificar la ruta de acceso en la instrucción que se muestra en este apartado. El archivo es .sav por lo cual en google colab se necesita la librería pyreadstat y la instrucción  pyreadstat.read_sav('ruta de acceso')
data, meta = pyreadstat.read_sav('/content/drive/MyDrive/ProyectoP2/dataRN.sav')

# Mostrar las primeras filas del DataFrame
data.head()

,AÑO,MES,DIASESTANCIA,SEXO,PPERTENENCIA,EDADAÑOS,DEPTORESIDEN,MUNIRESIDEN,CONDIEGRES,TRATARECIB
0,2009.0,1.0,2.0,1.0,6.0,1.0,7.0,0719,1.0,1.0
1,2009.0,1.0,1.0,1.0,6.0,47.0,16.0,1601,1.0,1.0
2,2009.0,7.0,1.0,1.0,6.0,34.0,13.0,1308,1.0,1.0
3,2009.0,6.0,2.0,1.0,7.0,56.0,16.0,1601,1.0,1.0
4,2009.0,6.0,1.0,1.0,6.0,1.0,16.0,1601,1.0,1.0


In [69]:
#Se revisa la información del dataset:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1712963 entries, 0 to 1712962
Data columns (total 10 columns):
 #   Column        Dtype  
---  ------        -----  
 0   AÑO           float64
 1   MES           float64
 2   DIASESTANCIA  float64
 3   SEXO          float64
 4   PPERTENENCIA  float64
 5   EDADAÑOS      float64
 6   DEPTORESIDEN  float64
 7   MUNIRESIDEN   object 
 8   CONDIEGRES    float64
 9   TRATARECIB    float64
dtypes: float64(9), object(1)
memory usage: 130.7+ MB


#Inyectar aleatoriedad

In [70]:
data = data.sample(frac = 1, random_state = 42).reset_index(drop=True)

#Variable SEXO
Vamos a aplicar redes neuronales para la variable Sexo considerando el año, días de estancia, la edad del paciente y tratamiento recibido.


In [72]:
# Se crea una copia del dataset original para no modificarlo:
dataS = data.copy()

In [73]:
#Para que los datos puedan ser analizados por el modelo de redes nueronales y obtener una predicción más coherente se procederá a cambiar la clasificación de la variable SEXO. Para este caso 1 (hombre) será 0 y 2 (mujer) será 1. Con esto podemos decir que el modelo nos predecirá si la persona con las características ingresadas es o no es mujer:

dataS['SEXO'] = dataS['SEXO'].replace({1: 0, 2: 1})

In [74]:
dataS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1712963 entries, 0 to 1712962
Data columns (total 10 columns):
 #   Column        Dtype  
---  ------        -----  
 0   AÑO           float64
 1   MES           float64
 2   DIASESTANCIA  float64
 3   SEXO          float64
 4   PPERTENENCIA  float64
 5   EDADAÑOS      float64
 6   DEPTORESIDEN  float64
 7   MUNIRESIDEN   object 
 8   CONDIEGRES    float64
 9   TRATARECIB    float64
dtypes: float64(9), object(1)
memory usage: 130.7+ MB


In [76]:
# Mostrar las primeras filas del DataFrame
dataS.head(20)

,AÑO,MES,DIASESTANCIA,SEXO,PPERTENENCIA,EDADAÑOS,DEPTORESIDEN,MUNIRESIDEN,CONDIEGRES,TRATARECIB
0,2018.0,3.0,1.0,0.0,1.0,68.0,14.0,1406,1.0,1.0
1,2022.0,5.0,2.0,1.0,4.0,26.0,9.0,0921,1.0,3.0
2,2022.0,1.0,2.0,0.0,4.0,1.0,13.0,1302,1.0,1.0
3,2011.0,9.0,2.0,0.0,7.0,53.0,1.0,0101,1.0,1.0
4,2019.0,5.0,1.0,1.0,4.0,22.0,10.0,1010,1.0,2.0
5,2012.0,7.0,1.0,1.0,7.0,46.0,1.0,0101,1.0,1.0
6,2015.0,1.0,1.0,1.0,7.0,20.0,1.0,0115,1.0,3.0
7,2015.0,6.0,4.0,1.0,7.0,79.0,1.0,0101,1.0,1.0
8,2017.0,4.0,2.0,1.0,7.0,0.0,12.0,1202,1.0,1.0
9,2009.0,6.0,2.0,1.0,7.0,0.0,9.0,0901,1.0,1.0


In [77]:
#Definición de variables dependientes e independientes
X = dataS[['AÑO', 'DIASESTANCIA','EDADAÑOS', 'TRATARECIB']]
y = dataS['SEXO']

In [78]:
#Definición de set de entreno y prueba
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [79]:
#Diseño de la red neuronal
model = Sequential()
model.add(Dense(16, input_dim = 4, activation = 'relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [80]:
#Compilación de la red
model.compile(loss ='binary_crossentropy', optimizer ='adam', metrics =['accuracy'])

In [81]:
#Entreno y testeo
model.fit(X_train, y_train, epochs = 35, batch_size = 128, validation_data=(X_test, y_test))

Epoch 1/35
10707/10707 ━━━━━━━━━━━━━━━━━━━━ 28s 2ms/step - accuracy: 0.5911 - loss: 0.7085 - val_accuracy: 0.5367 - val_loss: 0.9709
Epoch 2/35
10707/10707 ━━━━━━━━━━━━━━━━━━━━ 42s 3ms/step - accuracy: 0.5973 - loss: 0.6749 - val_accuracy: 0.3825 - val_loss: 1.3452
Epoch 3/35
10707/10707 ━━━━━━━━━━━━━━━━━━━━ 40s 2ms/step - accuracy: 0.5989 - loss: 0.6636 - val_accuracy: 0.3825 - val_loss: 1.9630
Epoch 4/35
10707/10707 ━━━━━━━━━━━━━━━━━━━━ 40s 2ms/step - accuracy: 0.5981 - loss: 0.6595 - val_accuracy: 0.3825 - val_loss: 2.4466
Epoch 5/35
10707/10707 ━━━━━━━━━━━━━━━━━━━━ 28s 3ms/step - accuracy: 0.5985 - loss: 0.6598 - val_accuracy: 0.5836 - val_loss: 0.6320
Epoch 6/35
10707/10707 ━━━━━━━━━━━━━━━━━━━━ 39s 2ms/step - accuracy: 0.6002 - loss: 0.6409 - val_accuracy: 0.6178 - val_loss: 0.6092
Epoch 7/35
10707/10707 ━━━━━━━━━━━━━━━━━━━━ 42s 2ms/step - accuracy: 0.6022 - loss: 0.6289 - val_accuracy: 0.6175 - val_loss: 0.7012
Epoch 8/35
10707/10707 ━━━━━━━━━━━━━━━━━━━━ 39s 2ms/step - accuracy: 

In [82]:
#Presición del modelo
loss, acc = model.evaluate(X_test, y_test)
print(acc*100)

10707/10707 ━━━━━━━━━━━━━━━━━━━━ 15s 1ms/step - accuracy: 0.6172 - loss: 0.6079
61.75491213798523


In [93]:
#Realizar predicciones
#Variables de entrada - AÑO, DIASESTANCIA, EDADAÑOS, TRATARECIB

sexpaciente = np.array([[2018, 1, 15, 2]])
p = model.predict(sexpaciente)
print(p)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
[[0.8098933]]


#Variable PPERTENENCIA

In [94]:
#Para aplicar el algoritmo a la variable PPERTENENCIA se filtra los datos para considerar únicamente los años de 2009 a 2017 debido al cambio de categorías a partir de 2018:

# Filtrar datos entre 2009 y 2017 (inclusive)
dataPP= data[(data['AÑO'] == 2009) & (data['AÑO'] <= 2017)]

# Ver las primeras filas del DataFrame filtrado
dataPP.head()


,AÑO,MES,DIASESTANCIA,SEXO,PPERTENENCIA,EDADAÑOS,DEPTORESIDEN,MUNIRESIDEN,CONDIEGRES,TRATARECIB
9,2009.0,6.0,2.0,2.0,7.0,0.0,9.0,0901,1.0,1.0
12,2009.0,4.0,1.0,1.0,7.0,36.0,5.0,0502,1.0,1.0
154,2009.0,4.0,5.0,2.0,6.0,44.0,9.0,0901,1.0,1.0
155,2009.0,9.0,1.0,1.0,7.0,16.0,13.0,1312,1.0,1.0
167,2009.0,4.0,1.0,2.0,6.0,20.0,16.0,1601,1.0,3.0


In [95]:
#Para que los datos puedan ser analizados por el modelo de redes nueronales y obtener una predicción más coherente se procederá a cambiar la clasificación de la variable PPERTENENCIA. Para este caso 6 (indígena) será 0 y 7 (no indígena) será 1. Con esto podemos decir que el modelo nos predecirá si la persona con las características ingresadas es o no considerado no indígena:
dataPP['PPERTENENCIA'] = dataPP['PPERTENENCIA'].replace({6: 0, 7: 1})

<ipython-input-95-963e83a948ae>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataPP['PPERTENENCIA'] = dataPP['PPERTENENCIA'].replace({6: 0, 7: 1})


In [96]:
dataPP.info()

<class 'pandas.core.frame.DataFrame'>
Index: 43919 entries, 9 to 1712945
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   AÑO           43919 non-null  float64
 1   MES           43919 non-null  float64
 2   DIASESTANCIA  43919 non-null  float64
 3   SEXO          43919 non-null  float64
 4   PPERTENENCIA  43919 non-null  float64
 5   EDADAÑOS      43919 non-null  float64
 6   DEPTORESIDEN  43919 non-null  float64
 7   MUNIRESIDEN   43919 non-null  object 
 8   CONDIEGRES    43919 non-null  float64
 9   TRATARECIB    43919 non-null  float64
dtypes: float64(9), object(1)
memory usage: 3.7+ MB


In [98]:
#Definición de variables dependientes e independientes
X1 = dataPP[['SEXO','EDADAÑOS', 'TRATARECIB', 'DEPTORESIDEN']]
y1 = dataPP['PPERTENENCIA']

In [99]:
#Definición de set de entreno y prueba
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test  = train_test_split(X1, y1, test_size = 0.2, random_state = 42)

In [100]:
#Diseño de la red neuronal
model = Sequential()
model.add(Dense(32, input_dim = 4, activation = 'relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [101]:
#Compilación de la red
model.compile(loss ='binary_crossentropy', optimizer ='adam', metrics =['accuracy'])

In [102]:
#Entreno y testeo
model.fit(X_train, y_train, epochs = 50, batch_size = 100, validation_data=(X_test, y_test))

Epoch 1/50
352/352 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.6493 - loss: 1.7470 - val_accuracy: 0.7659 - val_loss: 0.5311
Epoch 2/50
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7656 - loss: 0.5217 - val_accuracy: 0.7659 - val_loss: 0.5025
Epoch 3/50
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7628 - loss: 0.5004 - val_accuracy: 0.7659 - val_loss: 0.5022
Epoch 4/50
352/352 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7668 - loss: 0.4965 - val_accuracy: 0.7631 - val_loss: 0.5032
Epoch 5/50
352/352 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.7612 - loss: 0.4983 - val_accuracy: 0.7659 - val_loss: 0.4938
Epoch 6/50
352/352 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7628 - loss: 0.4963 - val_accuracy: 0.7658 - val_loss: 0.4953
Epoch 7/50
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7554 - loss: 0.5007 - val_accuracy: 0.7633 - val_loss: 0.4938
Epoch 8/50
352/352 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7626 - loss: 0.4903 - val_accuracy: 0.

In [103]:
#Presición del modelo
loss, acc = model.evaluate(X_test, y_test)
print(acc*100)

275/275 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7672 - loss: 0.4446
76.68488025665283


In [113]:
#Realizar predicciones
#Variables de entrada - SEXO,EDADAÑOS, TRATARECIB, DEPTORESIDEN

pueblopertenece = np.array([[2, 31, 3, 1]])
p = model.predict(pueblopertenece)
print(p)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
[[0.9618514]]
